In [1]:
# Importing Libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [3]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [4]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [5]:
# Utility function to read the data from csv file
from ww import f
def _read_csv(filename):
    return pd.read_csv(filename, encoding='utf-8',delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f('har/{subset}/{signal}_{subset}.txt')
        signals_data.append(
            _read_csv((filename)).as_matrix()
        )
        #c=pd.read_csv(io.StringIO(filename).decode("UTF-8")).as_matrix()
        #signals_data.append(c)
        

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [6]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f('har/y_{subset}.txt')
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [7]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [8]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

In [9]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [10]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [11]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [21]:
# Initializing parameters
epochs = 10
batch_size = 16
n_hidden = 32

In [22]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [14]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [31]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(X_test.shape)


128
9
(2947, 128, 9)


### Hyperparameter tunning of 2 layer LSTM Architecture.

"Hyperas" are recommended to perform hyperparameter tuning of deeplearning models.because of less computation power and run time complexity here iam performing hyperparameter tuning with random values of number of neurons and dropout rate.

In [ ]:

# Source: https://towardsdatascience.com/a-guide-to-an-efficient-way-to-build-neural-network-architectures-part-i-hyper-parameter-8129009f131b
"""
from keras.utils import to_categorical
from hyperopt import Trials, STATUS_OK, tpe, rand
from hyperas import optim
from hyperas.distributions import choice, uniform, conditional
from keras import optimizers

# one hot encode
y_train = to_categorical(Y_train)
y_test= to_categorical(Y_test)

def data():

    return X_train, y_train, X_val, y_val

def create_model(X_train, y_train, X_test,y_test):
    from keras import models
    from keras import layers
    import numpy as np

    model = models.Sequential()
    model.add(LSTM({{choice([np.power(2, 3), np.power(2, 4), np.power(2, 5)])}}))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(LSTM({{choice([np.power(2, 3), np.power(2, 4), np.power(2, 5)])}}))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(layers.Dense(6, activation='sigmoid'))

    from keras import callbacks
        
    reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                  patience=5, min_lr=0.001)


    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    model.fit(X_train,
              y_train,
              epochs=10,
              batch_size=16,
              validation_data=(X_test,y_test)
              callbacks=[reduce_lr])

    score, acc = model.evaluate(X_test,y_test, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}


if __name__ == '__main__':

    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=15,
                                          trials=Trials())
    X_train, X_test, y_train, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
"""

In [29]:
def create_model(neurons,dropout_rate):
    model = Sequential()
# Configuring the parameters
    model.add(LSTM(neurons, input_shape=(timesteps, input_dim),return_sequences=True))
# Adding a dropout layer
    model.add(Dropout(dropout_rate))
    model.add(LSTM(neurons))
    model.add(Dropout(dropout_rate))
# Adding a dense output layer with sigmoid activation
    model.add(Dense(n_classes, activation='softmax'))
    print(model.summary())
    
    model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
    
    # Training the model
    model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)
    print("-------------confusion matrix----------------")
    print(confusion_matrix(Y_test, model.predict(X_test)))
    
    score = model.evaluate(X_test, Y_test)
    print("-------------score-----------------")
    print(score)



In [30]:
neurons=[64,84,94,128]
dropout_rate=[0.25,0.5,0.75]

for neu in neurons:
    for drop in dropout_rate:
        print("*************MODEL WITH NEURONS {0} AND DROPOUT RATE {1} **************".format(neu,drop))
        create_model(neu,drop)

*************MODEL WITH NEURONS 64 AND DROPOUT RATE 0.25 **************
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_31 (LSTM)               (None, 128, 64)           18944     
_________________________________________________________________
dropout_16 (Dropout)         (None, 128, 64)           0         
_________________________________________________________________
lstm_32 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dropout_17 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_16 (Dense)             (None, 6)                 390       
Total params: 52,358
Trainable params: 52,358
Non-trainable params: 0
_________________________________________________________________
None
Train on 7352 samples, validate on 2947 samples
Epoch 1/10
73

Train on 7352 samples, validate on 2947 samples
Epoch 1/10
7352/7352 [==============================] - 119s 16ms/step - loss: 1.0555 - acc: 0.5574 - val_loss: 0.8416 - val_acc: 0.6098
Epoch 2/10
7352/7352 [==============================] - 113s 15ms/step - loss: 0.6376 - acc: 0.7758 - val_loss: 0.6005 - val_acc: 0.7815
Epoch 3/10
7352/7352 [==============================] - 110s 15ms/step - loss: 0.3998 - acc: 0.8802 - val_loss: 0.5929 - val_acc: 0.8229
Epoch 4/10
7352/7352 [==============================] - 107s 15ms/step - loss: 0.2642 - acc: 0.9207 - val_loss: 0.5271 - val_acc: 0.8612
Epoch 5/10
7352/7352 [==============================] - 106s 14ms/step - loss: 0.2177 - acc: 0.9257 - val_loss: 0.3139 - val_acc: 0.9023
Epoch 6/10
7352/7352 [==============================] - 110s 15ms/step - loss: 0.2092 - acc: 0.9357 - val_loss: 0.3493 - val_acc: 0.8982
Epoch 7/10
7352/7352 [==============================] - 112s 15ms/step - loss: 0.1934 - acc: 0.9320 - val_loss: 0.3403 - val_acc: 

Epoch 8/10
7352/7352 [==============================] - 142s 19ms/step - loss: 0.1423 - acc: 0.9480 - val_loss: 0.3122 - val_acc: 0.9121
Epoch 9/10
7352/7352 [==============================] - 141s 19ms/step - loss: 0.1592 - acc: 0.9434 - val_loss: 0.2990 - val_acc: 0.9145
Epoch 10/10
7352/7352 [==============================] - 137s 19ms/step - loss: 0.1449 - acc: 0.9444 - val_loss: 0.3057 - val_acc: 0.9121
-------------confusion matrix----------------
Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  6      400        84        0                   0   
STANDING                 0      111       419        2                   0   
WALKING                  0        3         0      462                  22   
WALKING_DOWNSTAIRS       0        0         0        0                 420

2947/2947 [==============================] - 5s 2ms/step
-------------score-----------------
[0.39325434634627854, 0.8944689514760774]
*************MODEL WITH NEURONS 94 AND DROPOUT RATE 0.5 **************
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_45 (LSTM)               (None, 128, 94)           39104     
_________________________________________________________________
dropout_30 (Dropout)         (None, 128, 94)           0         
_________________________________________________________________
lstm_46 (LSTM)               (None, 94)                71064     
_________________________________________________________________
dropout_31 (Dropout)         (None, 94)                0         
_________________________________________________________________
dense_23 (Dense)             (None, 6)                 570       
Total params: 110,738
Trainable params: 110,738
Non-trainable params

Train on 7352 samples, validate on 2947 samples
Epoch 1/10
7352/7352 [==============================] - 172s 23ms/step - loss: 0.8140 - acc: 0.6591 - val_loss: 0.6964 - val_acc: 0.7357
Epoch 2/10
7352/7352 [==============================] - 166s 23ms/step - loss: 0.3523 - acc: 0.8708 - val_loss: 0.3577 - val_acc: 0.8812
Epoch 3/10
7352/7352 [==============================] - 169s 23ms/step - loss: 0.1969 - acc: 0.9289 - val_loss: 0.5451 - val_acc: 0.8626
Epoch 4/10
7352/7352 [==============================] - 169s 23ms/step - loss: 0.1716 - acc: 0.9380 - val_loss: 0.4451 - val_acc: 0.8782
Epoch 5/10
7352/7352 [==============================] - 167s 23ms/step - loss: 0.1555 - acc: 0.9415 - val_loss: 0.3829 - val_acc: 0.8968
Epoch 6/10
7352/7352 [==============================] - 168s 23ms/step - loss: 0.1447 - acc: 0.9426 - val_loss: 0.4820 - val_acc: 0.9016
Epoch 7/10
7352/7352 [==============================] - 171s 23ms/step - loss: 0.1534 - acc: 0.9430 - val_loss: 0.5025 - val_acc: 

Epoch 8/10
7352/7352 [==============================] - 164s 22ms/step - loss: 0.1712 - acc: 0.9429 - val_loss: 0.3197 - val_acc: 0.9118
Epoch 9/10
7352/7352 [==============================] - 168s 23ms/step - loss: 0.1689 - acc: 0.9412 - val_loss: 0.3785 - val_acc: 0.9094
Epoch 10/10
7352/7352 [==============================] - 167s 23ms/step - loss: 0.1465 - acc: 0.9446 - val_loss: 0.6970 - val_acc: 0.8989
-------------confusion matrix----------------
Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 537        0         0        0                   0   
SITTING                  0      427        38       24                   0   
STANDING                 0      124       396       10                   0   
WALKING                  0        0         0      468                   8   
WALKING_DOWNSTAIRS       0        0         0        9                 398

##### number of neurons=94
##### dropout rate = 0.25 are the best values which gives maximum accuracy

# Conclusion:

- Performed Hyperparameter tuning of LSTM model w.r.t number of neurons and dropout rate.
- Observed that with number of neurons=94 and dropout rate=0.25 gives maximum accuracy of 91.58%.
- With a 2 layer LSTM architecture we got 91.58% accuracy and a loss of 0.34.